In [1]:
!pip install -r requirements.txt

  Using cached text_hammer-0.1.5-py3-none-any.whl (7.6 kB)
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/d6/9e/8afc618cfed4b5dc602b11754d4d9193a268439704defae301bffca7f04c/spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata
  Using cached spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/9e/b8/ed5f794359d05cd0bffb894c6418da87b93016ee17b669d55c45d1bd5d5b/tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Using cached datasets-2.14.5-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages

In [2]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/kazanova/sentiment140")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading sentiment140.zip to .\sentiment140


100%|██████████| 80.9M/80.9M [00:04<00:00, 20.3MB/s]


In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\kausikdevanathan\DataspellProjects\SIH\sentimentAnalysis\sentiment140\training.1600000.processed.noemoticon.csv",names=["sentiment","id","date","query","user","text"],encoding='latin-1')
df.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
df.isnull().sum()

sentiment    0
id           0
date         0
query        0
user         0
text         0
dtype: int64

In [3]:
df.drop(["id","date","query","user"],axis=1,inplace=True)

In [4]:
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
import transformers
from transformers import AutoTokenizer,pipeline
model_path = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [6]:
sentiment("I am very happy")

[{'label': 'LABEL_2', 'score': 0.9876530170440674}]

In [7]:
sentiment("I am very sad")

[{'label': 'LABEL_0', 'score': 0.949698805809021}]

In [8]:
sentiment("I am very angry")

[{'label': 'LABEL_0', 'score': 0.9702324867248535}]

In [9]:
!python -m spacy download en_core_web_sm -q

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
# %%time
# import text_hammer as th
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()
# def text_preprocessing(df,col_name):
#     column = col_name
#     df[column] = df[column].progress_apply(lambda x:str(x).lower())
#     #     df[column] = df[column].progress_apply(lambda x: th.cont_exp(x))
#     #you're -> you are; i'm -> i am
#     df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
#     df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
#     df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
#     df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
#     df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
#     return(df)
# 
# df['text'] = text_preprocessing(df,'text')

In [11]:
# import multiprocessing
# import time
# 
# def text_preprocessing_worker(text):
#     """Preprocess text using a specific function."""
# 
#     # Preprocess the text using your chosen function
#     preprocessed_text = th.cont_exp(text)
# 
#     return preprocessed_text
# 
# # Create a pool of workers to execute the text preprocessing function in parallel
# start = time.time()
# pool = multiprocessing.Pool(4)
# 
# # Apply the text preprocessing function to each text in the DataFrame using the pool of workers
# preprocessed_texts = pool.map(text_preprocessing_worker, df['text'])
# 
# # Add the preprocessed texts to the DataFrame
# df['text'] = preprocessed_texts
# 
# # Close the pool of workers
# pool.close()
# print(time.time() - start)


In [12]:
%%time
import unicodedata
from tqdm._tqdm_notebook import tqdm_notebook

def vectorized_text_preprocessing_tqdm(df, col_name):
    """
    Performs text preprocessing on a Pandas DataFrame column using vectorized operations and shows the progress bar.

    Args:
        df: A Pandas DataFrame.
        col_name: The name of the column to preprocess.

    Returns:
        A Pandas DataFrame with the preprocessed column.
    """

    # Show the progress bar.
    tqdm_notebook.pandas()

    # Convert the column to lowercase.
    df[col_name] = df[col_name].progress_apply(lambda x: x.lower())

    # Remove emails, HTML tags, special characters, and accented characters.
    df[col_name] = df[col_name].progress_apply(lambda x: x.replace(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", ""))
    df[col_name] = df[col_name].progress_apply(lambda x: x.replace(r"<[^>]*>", ""))
    df[col_name] = df[col_name].progress_apply(lambda x: x.replace(r"[^\w\s]", ""))
    df[col_name] = df[col_name].progress_apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", errors="ignore").decode("ascii"))

    # Make the text base form.
    df[col_name] = df[col_name].progress_apply(lambda x: " ".join(x.split()))

    return df



# Preprocess the text column using the vectorized approach with progress bar.
X_train = vectorized_text_preprocessing_tqdm(df, "text")

<timed exec>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

CPU times: total: 15.2 s
Wall time: 15.2 s


In [13]:
type(X_train)

pandas.core.frame.DataFrame

In [14]:
X_train.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [15]:
X_train.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\SIH\sentimentAnalysis\sentiment140\preprocessed.csv")

In [16]:
X_train['sentiment'].value_counts()

sentiment
0    800000
4    800000
Name: count, dtype: int64

In [17]:
import datasets

dataset = datasets.load_dataset("csv", data_files={"train": "train.csv", "validation": "validation.csv", "test": "test.csv"}) 

FileNotFoundError: Unable to find 'C:/Users/kausikdevanathan/DataspellProjects/SIH/sentimentAnalysis\train.csv'